# Getting Started with COMPAS TNA

`compas_tna` is an implementation of Thrust Network Analysis (TNA) based on COMPAS. TNA is a constrained version of the force density method for form finding of compression-only force networks in equilibrium with vertical loads applied to the nodes of the network, and a fixed horizontal projection.

For more information about `compas_tna`, please refer to the Github repo and the online documentation:

* [Github repo](https://github.com/blockresearchgroup/compas_tna)
* [Documentation](https://blockresearchgroup.github.io/compas_tna/latest)

For a more advanced implementation of TNA as a Rhino Plugin with a proper UI, please check RhinoVAULT. Getting Started instructions are available in this [Gitbook]().

## Installation

This notebook provides a basic example of working with `compas_tna` directly in your browser using JupyterLite.

The basic JupyterLite packages are already pre-installed. However, the COMPAS packages still have to be installed "on-the-fly". Therefore, it is recommended that you run the cells one by one instead of all cells at once.

Especially the cell below should be run first and be allowed to complete, before running any other cells. Otherwise, you will likely encounter an error related to COMPAS packages not being found...

In [ ]:
%pip install -q jsonschema networkx numpy scipy pythreejs
%pip install -q compas>=2.8 --no-deps
%pip install -q compas_notebook>=0.10.0 --no-deps
%pip install -q compas_fd --no-deps
%pip install -q compas_tna>=0.6.0 --no-deps
%pip install -q compas_skeleton>=2.0.1 --no-deps

## The Form Diagram

The first step in a TNA form finding process is to create a form diagram. Here, we will create a diagram using `compas_skeleton`.

In [29]:
from compas_skeleton.datastructures import Skeleton

lines = [
    ([-1, 0, 0], [1, 0, 0]),
    ([1, 0, 0], [2, 1, 0]),
    ([1, 0, 0], [2, -1, 0]),
    ([-1, 0, 0], [-2, 1, 0]),
    ([-1, 0, 0], [-2, -1, 0]),
]

skeleton = Skeleton(lines)
skeleton.density = 2
skeleton.leaf_angle = 20

In [30]:
from compas_tna.diagrams import FormDiagram

formdiagram = FormDiagram.from_mesh(skeleton.pattern)

## Boundary Conditions

In [31]:
from compas.itertools import flatten

supports = []

for node in skeleton.graph.leaves():
    nbr = None
    for test in skeleton.pattern.vertex_neighbors(node):
        if skeleton.pattern.is_vertex_on_boundary(test):
            nbr = test
            break
    if nbr is not None:
        edges = skeleton.pattern.edge_loop((node, nbr))
        vertices = list(set(list(flatten(edges))))
        supports += vertices

formdiagram.vertices_attribute("is_support", True, keys=supports)

formdiagram.update_boundaries()

## The Force Diagram

In [32]:
from compas_tna.diagrams import ForceDiagram

forcediagram = ForceDiagram.from_formdiagram(formdiagram)
forcediagram.edges_attribute(name="lmin", value=0.5)

## Horizontal Equilibrium

In [33]:
from compas_tna.equilibrium import horizontal_nodal

horizontal_nodal(form=formdiagram, force=forcediagram, kmax=500)

## Vertical Equilibrium

In [ ]:
from compas_tna.equilibrium import vertical_from_zmax

vertical_from_zmax(formdiagram, zmax=0.7)

## Visualisation

In [ ]:
from compas_notebook import Viewer

viewer = Viewer()

viewer.scene.add(formdiagram, vertexsize=0.05)
# viewer.scene.add(forcediagram, show_vertices=False)

viewer.show()